In [1]:
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
import math
from sklearn.svm import SVR, LinearSVR, SVC, LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import f1_score
from sklearn.calibration import CalibratedClassifierCV

In [2]:
sys.path.insert(0, r'../../src')
from function_cache import *

In [3]:
T_0min = pd.read_csv('../../data/processed/price_before_close/T_0min.csv', index_col='Trading_Day', parse_dates=True)
TF_0min = pd.read_csv('../../data/processed/price_before_close/TF_0min.csv', index_col='Trading_Day', parse_dates=True)
T_5min = pd.read_csv('../../data/processed/price_before_close/T_5min.csv', index_col='Trading_Day', parse_dates=True)
TF_5min = pd.read_csv('../../data/processed/price_before_close/TF_5min.csv', index_col='Trading_Day', parse_dates=True)
T_10min = pd.read_csv('../../data/processed/price_before_close/T_10min.csv', index_col='Trading_Day', parse_dates=True)
TF_10min = pd.read_csv('../../data/processed/price_before_close/TF_10min.csv', index_col='Trading_Day', parse_dates=True)
T_15min = pd.read_csv('../../data/processed/price_before_close/T_15min.csv', index_col='Trading_Day', parse_dates=True)
TF_15min = pd.read_csv('../../data/processed/price_before_close/TF_15min.csv', index_col='Trading_Day', parse_dates=True)

In [4]:
T_features = pd.read_csv('../../data/processed/candidate_features/T_features.csv', index_col=0)
TF_features = pd.read_csv('../../data/processed/candidate_features/TF_features.csv', index_col=0)
T_spread_change = pd.read_csv('../../data/processed/different_ys/T_spread_change.csv', index_col=0)
TF_spread_change = pd.read_csv('../../data/processed/different_ys/TF_spread_change.csv', index_col=0)

In [5]:
features = pd.concat([T_features, TF_features])
spread_change = pd.concat([T_spread_change, TF_spread_change])

In [6]:
spread_sign = np.sign(spread_change)

X = features.values
y = spread_sign['0min_0min'].values

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [7]:
def f_importances(coef, names):
    imp = coef
    imp, names = zip(*sorted(zip(imp,names)))
    plt.barh(range(len(names)), imp, align='center')
    plt.yticks(range(len(names)), names)
    plt.show()

In [8]:
plt.rcParams['figure.figsize'] = [25, 15]

In [10]:
C_list = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05,  0.1, 1, 10]

In [11]:
score_df = pd.DataFrame()

for j in range(300):
    n = 5
    kf = KFold(n_splits=n, shuffle=True)
    f_score = pd.DataFrame(index=range(n), columns=C_list)
    #score = pd.DataFrame(index=range(n), columns=C_list)
    i = 0

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        for C in C_list:
            svc = LinearSVC(max_iter=10000, C=C)
            svc.fit(X_train, y_train)

            y_pred = svc.predict(X_test)
            f_score.loc[i, C] = f1_score(y_test, y_pred)
            #score.loc[i, C] = svc.score(X_test, y_test)

        i = i + 1
    score_df = pd.concat([score_df, f_score.mean()], axis=1)

D:\anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to

In [12]:
score_df.mean(1)

0.0001     0.602707
0.0005     0.607862
0.0010     0.614139
0.0050     0.637070
0.0100     0.655562
0.0500     0.689410
0.1000     0.696394
1.0000     0.653169
10.0000    0.607775
dtype: float64

In [9]:
svc = SVC(kernel='linear', max_iter=10000, C=0.1, probability=True)
svc.fit(X, y)

SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=10000, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [11]:
svc.predict_proba(X)

array([[0.37948052, 0.62051948],
       [0.29543291, 0.70456709],
       [0.28648697, 0.71351303],
       [0.32019706, 0.67980294],
       [0.34414775, 0.65585225],
       [0.27951921, 0.72048079],
       [0.32023608, 0.67976392],
       [0.5       , 0.5       ],
       [0.39983595, 0.60016405],
       [0.40198457, 0.59801543],
       [0.32018067, 0.67981933],
       [0.33985198, 0.66014802],
       [0.31469017, 0.68530983],
       [0.32019482, 0.67980518],
       [0.32017302, 0.67982698],
       [0.27878817, 0.72121183],
       [0.5       , 0.5       ],
       [0.5       , 0.5       ],
       [0.5       , 0.5       ],
       [0.5       , 0.5       ],
       [0.41230921, 0.58769079],
       [0.4712593 , 0.5287407 ],
       [0.31876934, 0.68123066],
       [0.32019272, 0.67980728],
       [0.32023268, 0.67976732],
       [0.29075054, 0.70924946],
       [0.29305292, 0.70694708],
       [0.4341413 , 0.5658587 ],
       [0.32017379, 0.67982621],
       [0.32318874, 0.67681126],
       [0.

In [12]:
svc.predict(X)

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1., -1., -1., -1., -1., -1., -1.,  1.,  1.,  1.,  1.,
        1., -1.,  1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1.,  1.])

In [13]:
linear_svc = LinearSVC(max_iter=10000, C=0.1)
linear_svc.fit(X, y)

LinearSVC(C=0.1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=10000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [14]:
linear_svc.predict(X)

array([-1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1.,  1.,
        1.,  1.,  1., -1., -1., -1., -1., -1., -1.,  1.,  1.,  1.,  1.,
        1., -1.,  1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1.,  1.])

In [16]:
coef_sr = pd.Series(np.abs(linear_svc.coef_[0]), index=features.columns)
coef_sr.sort_values()

10d_current_spread                0.004749
-1d_corr_act_price_def_OI         0.009946
-1d_corr_def_price_def_OI         0.012009
-1d_corr_def_volume_def_OI        0.013596
-1d_corr_act_OI_def_OI            0.018950
15d_spread_change                 0.037514
-1d_corr_def_price_def_volume     0.057188
-1d_corr_act_price_def_volume     0.060845
-1d_corr_def_volume_spread        0.064089
-1d_corr_act_volume_def_OI        0.091337
10d_act_price_exp_mva             0.095161
-1d_std                           0.096583
10d_current_price                 0.099777
-1d_corr_act_OI_def_volume        0.118127
-1d_corr_act_OI_spread            0.122577
-1d_corr_def_price_spread         0.136681
1d_z_diff_act_volume_act_OI       0.155093
5d_spread_change                  0.175974
10d_spread_change                 0.202845
-1d_corr_def_OI_spread            0.207276
-1d_corr_act_price_act_OI         0.210399
10d_spread_std_divided_by_std     0.224850
-1d_corr_act_volume_def_volume    0.234467
20d_spread_

In [17]:
coef_sr.to_csv('SVC.csv', header=True)